# TF в Colab

In [0]:
%config IPCompleter.greedy=True  # более лучший автокомплит

In [7]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf 
import numpy as np
print(tf.__version__)
tf.compat.v1.disable_eager_execution()

2.5.0


In [8]:
tf.reset_default_graph()

a = tf.placeholder(tf.float32, (2, 2))
b = tf.Variable(tf.ones((2, 2)))
c = a @ b

In [9]:
print(c)

Tensor("matmul:0", shape=(2, 2), dtype=float32)


In [10]:
print(list(tf.get_default_graph().get_operations()[-1].inputs))
print(list(tf.get_default_graph().get_operations()[-1].outputs))

[<tf.Tensor 'Placeholder:0' shape=(2, 2) dtype=float32>, <tf.Tensor 'matmul/ReadVariableOp:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'matmul:0' shape=(2, 2) dtype=float32>]


In [11]:
s = tf.InteractiveSession()

Metal device set to: Apple M1


2022-02-15 21:59:29.723480: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-15 21:59:29.723890: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
s.run(tf.global_variables_initializer())
s.run(c, feed_dict={a: np.ones((2, 2))})

array([[2., 2.],
       [2., 2.]], dtype=float32)

In [13]:
s.close()

# Оптимизация

In [14]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32, trainable=True)
f = x ** 2

In [15]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f, var_list=[x])

In [16]:
tf.trainable_variables()

[<tf.Variable 'x:0' shape=() dtype=float32>]

In [17]:
with tf.Session() as s:  # сессия сама закроется
    s.run(tf.global_variables_initializer())
    for i in range(10):
        _, curr_x, curr_f = s.run([step, x, f])
        print(curr_x, curr_f)

2022-02-15 21:59:44.361564: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-15 21:59:44.361618: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-15 21:59:44.366467: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-15 21:59:44.367879: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


0.35074174 0.12301977
0.2805934 0.078732654
0.22447471 0.0503889
0.17957976 0.032248892
0.14366381 0.02063929
0.11493105 0.0132091455
0.091944836 0.0084538525
0.07355587 0.0054104663
0.058844697 0.0034626983
0.047075756 0.0022161268


2022-02-15 21:59:44.608889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


# TensorBoard логирование

In [18]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2

In [19]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [20]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all()

In [21]:
with tf.Session() as s:
  summary_writer = tf.summary.FileWriter("logs/1", s.graph)
  s.run(tf.global_variables_initializer())
  for i in range(10):
      _, curr_summaries = s.run([step, summaries])
      summary_writer.add_summary(curr_summaries, i)
      summary_writer.flush()

2022-02-15 21:59:53.000331: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-15 21:59:53.000378: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-15 21:59:53.014377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-15 21:59:53.028062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Локально можно запустить  `tensorboard --logdir=./logs` в консоли.

А вот так можно запустить прямо в Google Colab:

In [22]:
! wget https://raw.githubusercontent.com/ZEMUSHKA/skillfactory-dl/master/setup_google_colab.py -O setup_google_colab.py -q
import setup_google_colab

# запускаем сервер tensorboard в фоне
import os
os.system("tensorboard --logdir=./logs --host 0.0.0.0 --port 6006 &")

# открываем порт из Google Colab
setup_google_colab.expose_port_on_colab(6006)

Port 6006 is closed, retrying...


sh: apt-get: command not found


Port 6006 is closed, retrying...


TensorBoard 2.6.0 at http://0.0.0.0:6006/ (Press CTRL+C to quit)


Port 6006 is closed, retrying...
Port 6006 is closed, retrying...
Port 6006 is closed, retrying...
Port 6006 is closed, retrying...
Port 6006 is closed, retrying...
Port 6006 is closed, retrying...
Port 6006 is closed, retrying...
Port 6006 is closed, retrying...
Port 6006 is closed, retrying...


KeyboardInterrupt: 

# Обучение линейной модели

In [23]:
# модельные данные
N = 1000
D = 3
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T)

(1000, 3) (1000, 1)
[[0.25447726 0.00566955 0.36972546]]


In [24]:
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=(None, D), name="features")
target = tf.placeholder(tf.float32, shape=(None, 1), name="target")

weights = tf.get_variable("weights", shape=(D, 1), dtype=tf.float32)
predictions = features @ weights


loss = tf.reduce_mean((target - predictions) ** 2, name="loss")

print(target.shape, predictions.shape, loss.shape)

(None, 1) (None, 1) ()


In [25]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

In [26]:
# производные это часть графа
tf.get_default_graph().get_operations()

[<tf.Operation 'features' type=Placeholder>,
 <tf.Operation 'target' type=Placeholder>,
 <tf.Operation 'weights/Initializer/random_uniform/shape' type=Const>,
 <tf.Operation 'weights/Initializer/random_uniform/min' type=Const>,
 <tf.Operation 'weights/Initializer/random_uniform/max' type=Const>,
 <tf.Operation 'weights/Initializer/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'weights/Initializer/random_uniform/sub' type=Sub>,
 <tf.Operation 'weights/Initializer/random_uniform/mul' type=Mul>,
 <tf.Operation 'weights/Initializer/random_uniform' type=Add>,
 <tf.Operation 'weights' type=VarHandleOp>,
 <tf.Operation 'weights/IsInitialized/VarIsInitializedOp' type=VarIsInitializedOp>,
 <tf.Operation 'weights/Assign' type=AssignVariableOp>,
 <tf.Operation 'weights/Read/ReadVariableOp' type=ReadVariableOp>,
 <tf.Operation 'matmul/ReadVariableOp' type=ReadVariableOp>,
 <tf.Operation 'matmul' type=MatMul>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'pow/y' type=Const>,
 

In [27]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for i in range(300):
        _, curr_loss, curr_weights = s.run([step, loss, weights], 
                                           feed_dict={features: x, target: y})
        if i % 50 == 0:
            print(curr_loss)

2022-02-15 22:00:59.573989: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-15 22:00:59.574063: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-15 22:00:59.579060: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-15 22:00:59.624827: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


0.116178855
0.052195497
0.041217502
0.03926909
0.03891988
0.038856618


In [28]:
# результат обучения
curr_weights.T

array([[ 0.25537354, -0.03024207,  0.38736644]], dtype=float32)

In [29]:
# правильные веса
w.T

array([[0.25447726, 0.00566955, 0.36972546]])

# Обучение в Keras

In [30]:
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
print(keras.__version__)

2.5.0


In [31]:
K.clear_session()  # очищает граф и перезапускает сессию

model = M.Sequential()
model.add(L.Dense(1, use_bias=False, input_shape=(D,)))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

In [33]:
model.fit(x, y, batch_size=8, epochs=10, callbacks=[keras.callbacks.TensorBoard("./logs/keras")])

Train on 1000 samples
Epoch 1/10


2022-02-15 22:01:10.968260: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-15 22:01:10.968289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-15 22:01:10.973133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-15 22:01:10.978163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-15 22:01:10.988571: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-15 22:01:10.99

 568/1000 [================>.............] - ETA: 0s - loss: 0.3030 - mse: 0.3030

2022-02-15 22:01:11.357306: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-02-15 22:01:11.357314: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-02-15 22:01:11.360348: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-02-15 22:01:11.363994: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-02-15 22:01:11.368498: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/keras/plugins/profile/2022_02_15_22_01_11
2022-02-15 22:01:11.369422: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/keras/plugins/profile/2022_02_15_22_01_11/Johns-MacBook-Pro.local.trace.json.gz
2022-02-15 22:01:11.372239: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/keras/plugins/profile/2022_02_15_22_01_11
2022-02-15 22:

1000/1000 [==============================] - 1s 624us/sample - loss: 0.2515 - mse: 0.2515
Epoch 2/10
1000/1000 [==============================] - 0s 243us/sample - loss: 0.1357 - mse: 0.1357
Epoch 3/10
1000/1000 [==============================] - 0s 247us/sample - loss: 0.1039 - mse: 0.1039
Epoch 4/10
1000/1000 [==============================] - 0s 246us/sample - loss: 0.0831 - mse: 0.0831
Epoch 5/10
1000/1000 [==============================] - 0s 243us/sample - loss: 0.0689 - mse: 0.0689
Epoch 6/10
1000/1000 [==============================] - 0s 242us/sample - loss: 0.0593 - mse: 0.0593
Epoch 7/10
1000/1000 [==============================] - 0s 233us/sample - loss: 0.0527 - mse: 0.0527
Epoch 8/10
1000/1000 [==============================] - 0s 238us/sample - loss: 0.0483 - mse: 0.0483
Epoch 9/10
1000/1000 [==============================] - 0s 260us/sample - loss: 0.0453 - mse: 0.0453
Epoch 10/10
1000/1000 [==============================] - 0s 245us/sample - loss: 0.0432 - mse: 0.0432


In [34]:
s = K.get_session()
s.run(model.weights)

2022-02-15 22:01:16.208745: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


[array([[0.29914093],
        [0.09611366],
        [0.21689846]], dtype=float32)]